# Datan pika-analyysi Pythonilla

Näin saat frekvenssitaulukot, ristiintaulukoinnit ja keskeiset tunnusluvut nopeasti Exceliin.

Voit käyttää koodia mille tahansa datalle:
<ul><li>Vaihda avattavan datan tilalle oma data.</li>
    <li>Päivitä muuttujalistat (kategoriset, mielipiteet, kvantit ja monivalinnat) oman datasi
        mukaisiksi.</li></ul>
        
Koodin suorittamisen jälkeen tiedosto example1.csv löytyy samasta kansiosta kuin tämä koodi. Voit avata tiedostot Exceliin. Mahdolliset monivalinnat tallentuvat toiseen tiedostoon example2.csv.

Excelissä kannattaa ensimmäisenä vaihtaa desimaalilukuna esitetyt prosentit prosenttimuotoon.

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('http://taanila.fi/data1.xlsx')
df.head()

,nro,sukup,ikä,perhe,koulutus,palveluv,palkka,johto,työtov,työymp,palkkat,työteht,työterv,lomaosa,kuntosa,hieroja
0,1,1,38,1,1.0,22.0,3587,3,3.0,3,3,3,NaN,NaN,NaN,NaN
1,2,1,29,2,2.0,10.0,2963,1,5.0,2,1,3,NaN,NaN,NaN,NaN
2,3,1,30,1,1.0,7.0,1989,3,4.0,1,1,3,1.0,NaN,NaN,NaN
3,4,1,36,2,1.0,14.0,2144,3,3.0,3,3,3,1.0,NaN,NaN,NaN
4,5,1,24,1,2.0,4.0,2183,2,3.0,2,1,2,1.0,NaN,NaN,NaN


In [3]:
#määrittelen mitkä muuttujista ovat kategorisia, mielipideasteikollisia tai määrällisiä
#jos tietyn tyyppisiä muuttujia ei ole, niin käytän tyhjää listaa, esimerkiksi kvantit=[]
kategoriset = ['sukup','perhe','koulutus']
mielipiteet = ['johto','työtov','työymp','palkkat','työteht']
kvantit = ['ikä','palveluv','palkka']

In [4]:
#avaan/luon tiedoston, johon tulokset kirjoitetaan
#varoitus: Jos samanniminen tiedosto on jo olemassa, niin menetän sen
file1 = open('example1.csv', 'w', newline='')

#kategoristen muuttujien frekvenssitaulukot
#if-lause tarvitaan, jotta ei lähdeä tyhjän listan kanssa for-silmukkaan
if kategoriset:
    for var in kategoriset:
        df1=pd.crosstab(df[var],'Lkm')
        df1['Prosenttia'] = df1/df1.sum()
        df1.loc['Yhteensä'] = df1.sum()
        df1.to_csv(file1, sep=';', decimal=',')
        file1.write('\n')

#mielipideasteikollisten muuttujien frekvenssitaulukot
if mielipiteet:
    for var in mielipiteet:
        df1=pd.crosstab(df[var],'Lkm')    
        df1['Prosenttia'] = df1/df1.sum()
        df1.loc['Yhteensä'] = df1.sum()
        df1.to_csv(file1, sep=';', decimal=',')
        file1.write('\n')

#ristiintaulukoinnit kategoristen muuttujien välillä
if kategoriset:
    for var1 in kategoriset:
        for var2 in kategoriset:
            if var1!=var2:
                #Lukumäärä-taulukko:
                df1=pd.crosstab(df[var2],df[var1])
                df1.loc['Yhteensä']=df1.sum()
                #Prosentti-taulukko:
                df2=pd.crosstab(df[var2],df[var1],normalize='columns')
                df2.index.name=var2+'/'+var1
                df2.loc['Yhteensä']=df2.sum()
                df2.loc['n']=df1.loc['Yhteensä']
                df2.to_csv(file1, sep=';', decimal=',')
                file1.write('\n')

#kategoristen ja mielipideasteikollisten väliset ristiintaulukoinnit
if kategoriset and mielipiteet:
    for var1 in kategoriset:
        for var2 in mielipiteet:
            #Lukumäärä-taulukko:
            df1=pd.crosstab(df[var2],df[var1])
            df1.loc['Yhteensä']=df1.sum()
            #Prosentti-taulukko:
            df2=pd.crosstab(df[var2],df[var1],normalize='columns')
            df2.index.name=var2+'/'+var1
            df2.loc['Yhteensä']=df2.sum()
            df2.loc['n']=df1.loc['Yhteensä']
            df2.to_csv(file1, sep=';', decimal=',')
            file1.write('\n')

#mielipiteiden tunnusluvut
if mielipiteet:
    df1=df[mielipiteet].describe()
    df1.transpose().to_csv(file1, sep=';', decimal=',')
    file1.write('\n')

#ielipiteiden tunnusluvut kategoristen määräämissä ryhmissä
if kategoriset and mielipiteet:
    for var1 in kategoriset:
        for var2 in mielipiteet:
            df1=df.groupby(var1)[var2].describe()
            df1.index.name=var2+'/'+var1
            df1.to_csv(file1, sep=';', decimal=',')
            file1.write('\n')

#määrällisten muuttujien tunnusluvut
if kvantit:
    df1=df[kvantit].describe()
    df1.to_csv(file1, sep=';', decimal=',')
    file1.write('\n')

#määrällisten tunnusluvut kategoristen määräämissä ryhmissä
if kategoriset and kvantit:
    for var1 in kategoriset:
        for var2 in kvantit:
            df1=df.groupby(var1)[var2].describe()
            df1.index.name=var2+'/'+var1
            df1.to_csv(file1, sep=';', decimal=',')
            file1.write('\n')

#määrällisten völiset korrelaatiot
if kvantit:
    df1=df[kvantit].corr()
    df1.index.name='Korrelaatiot'
    df1.to_csv(file1, sep=';', decimal=',')
    file1.write('\n')

#suljen tiedoston
file1.close()

In [5]:
#määrittelen samaan monivalintakysymykseen liittyvät vaihtoehdot
#jos sinulla on useita monivalintakysymyksiä, niin tämä lohko pitää toistaa jokaiselle monivalintakysymykselle
monivalinnat=['työterv','lomaosa','kuntosa','hieroja']

#avaan/luon tiedoston, johon tulokset kirjoitetaan
#varoitus: jos samanniminen tiedosto on jo olemassa, niin menetän sen
file2=open('example2.csv', 'w', newline='')

#valinnat lukumäärinä ja prosentteina kyselyn vastaajista
df1 = df[monivalinnat].sum().sort_values(ascending = False).to_frame()
df1 = df1.rename(columns = {0: 'Lkm'})
df1['% vastaajista'] = df1['Lkm'] / df.shape[0]
df1.loc['n','% vastaajista']=df.shape[0]
df1.to_csv(file2, sep=';', decimal=',')
file2.write('\n')

#valinnat kategoristen määräämissä ryhmissä lukumäärinä ja prosentteina ryhmästä
for var1 in kategoriset:
    df1 = df.groupby(var1)[monivalinnat].sum()
    df1.to_csv(file2, sep=';', decimal=',')
    file2.write('\n')
    for value in df[var1].dropna().unique():
        value_total=df[var1].value_counts()[value]
        df1.loc[value]=df1.loc[value]/value_total
        df1.loc[value, 'n']=value_total
    df1.to_csv(file2, sep=';', decimal=',')
    file2.write('\n')
    
#suljen tiedoston
file2.close()